In [1]:
from Functions.documents import load_documents
from Functions.text import clear_text, remove_stopwords, split_text_into_chunks, remove_portuguese_accents



DOCUMENT_PATHS = ["./Documents/a (1).pdf", "./Documents/FAQ (1).pdf"]
CHUNK_SIZE = 500



# BEGIN = Processing documents 
texts = load_documents(DOCUMENT_PATHS)
cleaned_texts = [clear_text(text) for text in texts]
no_stopwords_text = [remove_stopwords(text) for text in cleaned_texts]
no_accents_portuguese = [remove_portuguese_accents(text) for text in no_stopwords_text]
joined_texts = " ".join(no_accents_portuguese)
joined_texts = joined_texts.lower()
text_chunks = split_text_into_chunks(joined_texts, chunk_size=CHUNK_SIZE)
# END = Processing documents 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
len(text_chunks)

22

In [3]:
len(text_chunks[0])

3894

import gym
import numpy as np
from gym import spaces
from sentence_transformers import SentenceTransformer, util
from Functions.documents import load_documents
from Functions.text import clear_text, remove_stopwords, split_text_into_chunks

In [6]:
model = SentenceTransformer("all-MiniLM-L6-v2")

c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\rafae\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [8]:
encoded = model.encode(text_chunks[0])

In [9]:
not_encoded = text_chunks[0]

In [10]:
len(encoded)

384

In [11]:
len(not_encoded)

3894

In [12]:
document_embeddings = [model.encode(chunk) for chunk in text_chunks] 

In [39]:
import textdistance
# https://github.com/life4/textdistance?tab=readme-ov-file

f1 = "o carro está andando"
# f2 = "o ônibus está em movimento"
f2 = "o ônibus andando"
print("monge: ",textdistance.monge_elkan(f1,f2))
print("cosine: ",textdistance.cosine(f1,f2))
print("hamming: ",textdistance.hamming.normalized_similarity(f1,f2))

monge:  0.034999999999999996
cosine:  0.6149186938124421
hamming:  0.09999999999999998
